# Boletin 5.4

### Ejercicios XPath y XQuery

**Entrega**

La entrega se realizará en este mismo documento incluyendo las consultas XPath y XQuery realizadas.

**Recursos**
- Apuntes: Para la realización de estos ejercicios utilizar la documentación colgada en el aula virtual (moodle) de la asignatura.
- [XML consultas](res/courses_xpath.xml):  Utilizar este fichero XML para realizar las consultas.


**Aclaraciones estructura XML**
- Es un catalogo de cursos, agrupados por departametos.
- Cada departamento, tiene un código, un nombre, y un profesor responsable (`chair`)
- Cada curso tiene un número, nº alumnos matriculados (`enrollment`), y un profesor responsable (`instructor`). Un curso puede tener un curso pre-requisito (`prereq`), que se debe realizar antes de poder matricularse en el curso.
- Los profesores (instuctores) pueden impartir clase (Profesor) o ser ayudantes (Lecturer).

### Ejercicios

1. Retorna todos los elementos títulos (de ambos departamentos y cursos)

In [ ]:
for $title in distinct-values(//Title)
return $title

In [ ]:
<Title>Computer Science</Title>
<Title>Programming Methodology</Title>
<Title>Programming Abstractions</Title>
<Title>Computer Organization and Systems</Title>
<Title>Introduction to Probability for Computer Scientists</Title>
<Title>From Languages to Information</Title>
<Title>Compilers</Title>
<Title>Introduction to Databases</Title>
<Title>Artificial Intelligence: Principles and Techniques</Title>
<Title>Structured Probabilistic Models: Principles and Techniques</Title>
<Title>Machine Learning</Title>
<Title>Electrical Engineering</Title>
<Title>Digital Systems I</Title>
<Title>Digital Systems II</Title>
<Title>Linguistics</Title>
<Title>From Languages to Information</Title>

2. Retorna los `last-names` de todos los departamentos `chairs`

In [ ]:
for $chair in distinct-values(//Department/Chair/Professor/Last_Name)
return $chair

In [ ]:
<Last_Name>Widom</Last_Name>
<Last_Name>Horowitz</Last_Name>
<Last_Name>Levin</Last_Name>

3. Retorna títulos de cursos con `enrollment` mayor que 500

In [ ]:
for $course in //Course[@Enrollment > 500]
return $course/Title

In [ ]:
<Title>Programming Abstractions</Title>
<Title>From Languages to Information</Title>
<Title>Compilers</Title>
<Title>From Languages to Information</Title>

4. Retorna títulos de departamentos que tengan algún curso que tiene `CS106B` como prerequisito

In [ ]:
for $department in //Department[Course/Prerequisites/Prereq = "CS106B"]
return $department/Title

In [ ]:
<Title>Computer Science</Title>
<Title>Electrical Engineering</Title>

5. Retorna `last-names` de todos los profesores y lecturers quién usa un `middle initial`.

In [ ]:
for $person in //(Professor | Lecturer)[Middle_Initial]
return $person/Last_Name

In [ ]:
<Last_Name>Cain</Last_Name>
<Last_Name>Cain</Last_Name>
<Last_Name>Aiken</Last_Name>
<Last_Name>Horowitz</Last_Name>
<Last_Name>Dally</Last_Name>

6. Retorna el número de cursos que tiene un `cross-listed` en su descripción

In [ ]:
let $catalog := //Course_Catalog
return count($catalog//Course[contains(Description, "Cross-listed")])

In [ ]:
2

7. Retorna la media de matriculados de todos los cursos en el departamento de `CS`

In [ ]:
let $courses := //Department[@Code = 'CS']/Course
let $totalEnrollments := sum($courses/@Enrollment)
let $numCourses := count($courses)
return $totalEnrollments div $numCourses

In [ ]:
336

8. Retonar los `last-names` de instructores que enseñan al menos un curso que tiene `system` en su descripción y `enrollment` mayor que 100

In [ ]:
distinct-values(
  for $course in //Course[contains(Description, "system") and @Enrollment > 100]
  for $instructor in $course/Instructors/*/Last_Name
  return $instructor
)

In [ ]:
<Last_Name>Widom</Last_Name>

9. Retorna el título del curso con el número de matriculados más alto

In [ ]:
let $maxEnrollment := max(//Course/@Enrollment)
return //Course[@Enrollment = $maxEnrollment]/Title

In [ ]:
<Title>Programming Methodology</Title>

10. Retorna el numero de cursos que tienen no `lecturers` como instructores

In [ ]:
let $numCourses := count(//Course[not(Instructors/Lecturer)])
return $numCourses

In [ ]:
10

11. Retornoa tiítulos de cursos impartidos por el `chair` del departamento.

In [ ]:
for $department in //Course_Catalog/Department
let $chairFirstName := $department/Chair/Professor/First_Name
let $chairLastName := $department/Chair/Professor/Last_Name
where $department/Course/Instructors/*/First_Name = $chairFirstName and $department/Course/Instructors/*/Last_Name = $chairLastName
return $department/Course/Title

In [ ]:
<Title>Programming Methodology</Title>
<Title>Programming Abstractions</Title>
<Title>Computer Organization and Systems</Title>
<Title>Introduction to Probability for Computer Scientists</Title>
<Title>From Languages to Information</Title>
<Title>Compilers</Title>
<Title>Introduction to Databases</Title>
<Title>Artificial Intelligence: Principles and Techniques</Title>
<Title>Structured Probabilistic Models: Principles and Techniques</Title>
<Title>Machine Learning</Title>

12. Retorna títulos de cursos impartidos por un profesor con el `last-name` ea "Ng" pero no por un profesor con el `last-name` "Thrun"  💡(usar count() = 0)

In [ ]:
for $course in //Course
where $course/Instructors/Professor[Last_Name = 'Ng'] and count($course/Instructors/Professor[Last_Name = 'Thrun']) = 0
return $course/Title

In [ ]:
<Title>Machine Learning</Title>

13. Haz un sumatorio de CS clases. Lista todos los departamentos `CS` en orden de número de alumnos matriculados.<br>
Por cada curso incluye solo su `Enrollement` (como atributo) y el `title` (como sub-elemento contenido).<br>
  
Ejemplo de salida:
  ```xml
  <Summary> 
    ...
    <Course Enrollment="60">
      <Title>From Languages to Information</Title>
    </Course>
    <Course Enrollment="90">
      <Title>Compilers</Title>
    </Course>
    ...
  </Summary>
    ```

In [ ]:
<Summary>
{
  for $department in //Course_Catalog/Department[@Code = 'CS']
  order by sum($department/Course/@Enrollment) descending
  return
  <Department Code="CS" TotalEnrollment="{sum($department/Course/@Enrollment)}">
  {
    for $course in $department/Course
    return
    <Course Enrollment="{data($course/@Enrollment)}">
      <Title>{data($course/Title)}</Title>
    </Course>
  }
  </Department>
}
</Summary>

In [ ]:
<Summary>
    <Department Code="CS" TotalEnrollment="3360">
        <Course Enrollment="1070">
            <Title>Programming Methodology</Title>
        </Course>
        <Course Enrollment="620">
            <Title>Programming Abstractions</Title>
        </Course><Course Enrollment="500">
            <Title>Computer Organization and Systems</Title>
        </Course>
        <Course Enrollment="280">
            <Title>Introduction to Probability for Computer Scientists</Title>
        </Course>
        <Course Enrollment="60"><
            Title>From Languages to Information</Title>
        </Course>
        <Course Enrollment="90">
            <Title>Compilers</Title>
        </Course>
        <Course Enrollment="130">
            <Title>Introduction to Databases</Title>
        </Course>
        <Course Enrollment="180">
            <Title>Artificial Intelligence: Principles and Techniques</Title>
        </Course>
        <Course Enrollment="110">
            <Title>Structured Probabilistic Models: Principles and Techniques</Title>
        </Course><Course Enrollment="320">
            <Title>Machine Learning</Title>
        </Course>
    </Department>
</Summary>

14. Retorna un `Profesores` elemento que contenga como sub-elementos un listado de todos los profesores in todos los departamentos, ordenados por `last-name` con cada profesor apareciendo una vez. <br>
El `Profesor` subelementos deberian tener la misma estructura como en el documento original

    ✏️ **nota**: La presencia o ausencia de un `middle-initial` puede requerir acciones para su manejo.
  
Ejemplo de salida:
  ```xml
  <Profesores>
    <Profesor>
      <First_Name>Alex</First_Name>
      <Middle_Initial>S.</Middle_Initial>
      <Last_Name>Aiken</Last_Name>    
    </Profesor>
    <Profesor>
      <First_Name>William</First_Name>
      <Middle_Initial>J.</Middle_Initial>
      <Last_Name>Dally</Last_Name>
    </Profesor>  
    ...
  </Profesores>
    ```

In [ ]:
<Profesores>{
  for $profesor in distinct-values(//Professor/concat(First_Name, " ", Last_Name))
  let $first_name := substring-before($profesor, " ")
  let $last_name := substring-after($profesor, " ")
  let $middle_initial := if (contains($profesor, " ")) then substring-before(substring-after($profesor, " "), ".") else ()
  order by $last_name
  return
  <Profesor>
    <First_Name>{$first_name}</First_Name>
    {
      if (exists($middle_initial))
      then <Middle_Initial>{$middle_initial}</Middle_Initial>
      else ()
    }
    <Last_Name>{$last_name}</Last_Name>
  </Profesor>
}</Profesores>

In [ ]:
<Profesores>
    <Profesor>
        <First_Name>Alex</First_Name>
        <Middle_Initial/>
        <Last_Name>Aiken</Last_Name>
    </Profesor>
    <Profesor><
        First_Name>William</First_Name>
        <Middle_Initial/>
        <Last_Name>Dally</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Mark</First_Name>
        <Middle_Initial/>
        <Last_Name>Horowitz</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Dan</First_Name>
        <Middle_Initial/>
        <Last_Name>Jurafsky</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Daphne</First_Name>
        <Middle_Initial/>
        <Last_Name>Koller</Last_Name>
    </Profesor>
    <Profesor><
        First_Name>Beth</First_Name>
        <Middle_Initial/>
        <Last_Name>Levin</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Subhasish</First_Name>
        <Middle_Initial/>
        <Last_Name>Mitra</Last_Name>
    /Profesor>
    <Profesor>
        <First_Name>Andrew</First_Name>
        <Middle_Initial/>
        <Last_Name>Ng</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Oyekunle</First_Name>
        <Middle_Initial/>
        <Last_Name>Olukotun</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Eric</First_Name>
        <Middle_Initial/>
        <Last_Name>Roberts</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Mehran</First_Name>
        <Middle_Initial/>
        <Last_Name>Sahami</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Sebastian</First_Name>
        <Middle_Initial/>
        <Last_Name>Thrun</Last_Name>
    </Profesor>
    <Profesor>
        <First_Name>Jennifer</First_Name>
        <Middle_Initial/>
        <Last_Name>Widom</Last_Name>
    </Profesor>
</Profesores>